In [133]:
import pandas as pd
import pickle
import numpy as np
import itertools

In [198]:
df_parts = pd.read_csv('data/parts.csv')
df_parts_categories = pd.read_csv('data/part_categories.csv')
df_inventories = pd.read_csv('data/inventories.csv')
df_inventories_parts = pd.read_csv('data/inventory_parts.csv')
df_sets = pd.read_csv('data/sets.csv')

In [204]:
df_PARTS = pd.merge(df_parts,df_parts_categories,left_on='part_cat_id',right_on='id',how='left',suffixes=['_part','_cat'])

In [205]:
df_PARTS.head()

,part_num,name_part,part_cat_id,id,name_cat
0,0687b1,Set 0687 Activity Booklet 1,17,17,Non-LEGO
1,0901,Baseplate 16 x 30 with Set 080 Yellow House Print,1,1,Baseplates
2,0902,Baseplate 16 x 24 with Set 080 Small White Hou...,1,1,Baseplates
3,0903,Baseplate 16 x 24 with Set 080 Red House Print,1,1,Baseplates
4,0904,Baseplate 16 x 24 with Set 080 Large White Hou...,1,1,Baseplates


In [216]:
df_SETS = pd.merge(df_inventories_parts,df_inventories,left_on='inventory_id',right_on='id',how='left')
df_SETS = pd.merge(df_SETS,df_sets,on='set_num',how='left')
df_SETS = pd.merge(df_SETS,df_PARTS[['part_num','name_part','name_cat']],on='part_num',how='left')
df_SETS.head()

,inventory_id,part_num,color_id,quantity,is_spare,id,version,set_num,name,year,theme_id,num_parts,name_part,name_cat
0,1,48379c01,72,1,f,1,1,7922-1,McDonald's Sports Set Number 6 - Orange Vest S...,2004,460,4,Sports Promo Figure Base with Feet,Minifig Accessories
1,1,48395,7,1,f,1,1,7922-1,McDonald's Sports Set Number 6 - Orange Vest S...,2004,460,4,Sports Promo Snowboard from McDonald's Sports ...,Minifig Accessories
2,1,mcsport6,25,1,f,1,1,7922-1,McDonald's Sports Set Number 6 - Orange Vest S...,2004,460,4,Sports Promo Figure Head Torso Assembly McDona...,Minifigs
3,1,paddle,0,1,f,1,1,7922-1,McDonald's Sports Set Number 6 - Orange Vest S...,2004,460,4,Sports Promo Paddle from McDonald's Sports Sets,Minifig Accessories
4,3,11816pr0005,78,1,f,3,1,3931-1,Emma's Splash Pool,2012,494,43,"Minifig Head Modified - Friends - Green Eyes, ...",Minifigs


### Drop the figurines from castle

In [301]:
df_6075_2_BOM = df_SETS[df_SETS.set_num=='6075-2']
df_6075_2_BOM.groupby('name_cat').count()
print df_6075_2_BOM.name_cat.unique()
CATEGORIES_TO_EXCLUDE = ['Flags, Signs, Plastics and Cloth','Minifigs','Minifig Accessories']

['Bricks' 'Plates' 'Bricks Sloped' 'Bricks Round and Cones' 'Tiles'
 'Windows and Doors' 'Plates Special' 'Bars, Ladders and Fences'
 'Bricks Curved' 'Bricks Special' 'Flags, Signs, Plastics and Cloth'
 'Minifigs' 'Technic Pins' 'Technic Bricks' 'Hinges, Arms and Turntables'
 'Minifig Accessories' 'Baseplates' 'String, Bands and Reels']


## Aggregate all colors quantities

In [303]:
df_6075_2_BOM = df_SETS[df_SETS.set_num=='6075-2']
df_6075_2_BOM = df_6075_2_BOM[~df_6075_2_BOM.name_cat.isin(CATEGORIES_TO_EXCLUDE)]
df_6075_2_BOM = df_6075_2_BOM.groupby(['part_num'])[['quantity']].sum().reset_index()
df_6075_2_BOM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 2 columns):
part_num    45 non-null object
quantity    45 non-null int64
dtypes: int64(1), object(1)
memory usage: 792.0+ bytes


In [304]:
df_SETS_BOM= df_SETS.groupby(['set_num','part_num','name'])[['quantity']].sum()

In [305]:
df_SETS_BOM.reset_index(inplace=True)

In [306]:
len(df_SETS_BOM.set_num.unique())

10713

In [384]:
df_BOMs = pd.DataFrame()
i=0
for name,group in df_SETS_BOM.groupby('set_num'):
    df = pd.merge(df_6075_2_BOM,group, on='part_num', how='inner',suffixes=['_6075_2','_sets'])
    
    df['Set']=name
    df_BOMs = pd.concat([df_BOMs,df],axis=0)
    i+=1
    if i%1000==0:
        print i

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [308]:
df_BOMs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69308 entries, 0 to 1
Data columns (total 4 columns):
quantity_6075_2    69308 non-null int64
set_num            69308 non-null object
quantity_sets      69308 non-null int64
Set                69308 non-null object
dtypes: int64(2), object(2)
memory usage: 2.6+ MB


In [309]:
df_BOMs.head()

,quantity_6075_2,set_num,quantity_sets,Set
0,12,00-1,22,00-1
1,16,00-1,12,00-1
2,195,00-1,115,00-1
3,73,00-1,57,00-1
4,15,00-1,18,00-1


In [387]:
df_BOMs['Filled_Quantity']=df_BOMs.quantity_6075_2-df_BOMs.quantity_sets
df_BOMs.Filled_Quantity = np.where(df_BOMs.Filled_Quantity<0,df_BOMs.quantity_6075_2,df_BOMs.quantity_sets)

In [311]:
df_BOMs.head()

,quantity_6075_2,set_num,quantity_sets,Set,Filled_Quantity
0,12,00-1,22,00-1,12
1,16,00-1,12,00-1,12
2,195,00-1,115,00-1,115
3,73,00-1,57,00-1,57
4,15,00-1,18,00-1,15


In [313]:
df_BOMs_grouped = df_BOMs.groupby('set_num')[['Filled_Quantity']].sum()

df_BOMs_grouped.sort_values(by='Filled_Quantity',ascending=False)
df_BOMs_grouped.reset_index(inplace=True)
print df_BOMs_grouped.shape
print len(df_BOMs_grouped.set_num.unique())
df_BOMs_grouped = pd.merge(df_BOMs_grouped,df_catalog[['set_num','year','name']].drop_duplicates(), left_on='set_num',right_on='set_num', how='left')

df_BOMs_grouped[df_BOMs_grouped.year>2006].sort_values(by='Filled_Quantity',ascending=False).head(10)

(7370, 2)
7370


,set_num,Filled_Quantity,year,name
127,10188-1,597,2008,Death Star
5599,75159-1,564,2016,Death Star
5666,75827-1,556,2016,Firehouse Headquarters
155,10224-1,527,2012,Town Hall
206,10654-1,516,2016,XL Creative Brick Box
215,10664-1,516,2013,Creative Tower
246,10697-1,511,2015,XXXL Box
3565,5512-1,486,2010,LEGO XXL Box
3608,5589-1,486,2009,Giant Box
128,10189-1,485,2008,Taj Mahal


In [316]:
TOP_SETS = df_BOMs_grouped[df_BOMs_grouped.year>2006].nlargest(20,'Filled_Quantity')

In [317]:
TOP_SETS.head()


,set_num,Filled_Quantity,year,name
127,10188-1,597,2008,Death Star
5599,75159-1,564,2016,Death Star
5666,75827-1,556,2016,Firehouse Headquarters
155,10224-1,527,2012,Town Hall
206,10654-1,516,2016,XL Creative Brick Box


In [356]:
TOP_SETS = ['10654-1']

### Select on set and compute missing parts


### Create a list of all possible 3 combinations

In [357]:
ALL_SETS_RECENTS = (df_catalog[df_catalog.year>2008]).set_num.unique()
len(ALL_SETS_RECENTS)

4337

In [359]:
ALL_SETS_NO_TOPS = list(set(ALL_SETS_RECENTS)-set(TOP_SETS))

In [361]:
COMBI_SETS = list(itertools.product(TOP_SETS,ALL_SETS_NO_TOPS ))
print len(COMBI_SETS)

4336


In [362]:
print len(COMBI_SETS)
print COMBI_SETS[0]

4336
('10654-1', '30201-1')


In [363]:
def combine_sets(df_SETS_BOM,sets_list):
    df = df_SETS_BOM[df_SETS_BOM.set_num.isin(sets_list)] 
    df = df.groupby('part_num')[['quantity']].sum()
    return df.reset_index()

In [364]:
TOTAL_CASTLE_PARTS = df_6075_2_BOM.quantity.sum()
print TOTAL_CASTLE_PARTS

676


In [365]:
def compute_combination_fullfillment_score(df_6075_2_BOM,df_set):
    df = pd.merge(df_6075_2_BOM,df_set, on='part_num',how='left',suffixes=['_6075_2','_sets'])
    df['Filled_Quantity']=df.quantity_6075_2-df.quantity_sets
    df.Filled_Quantity = np.where(df.Filled_Quantity<=0,df.quantity_6075_2,df.quantity_sets)
    df = df.fillna(0)
    
    return df.Filled_Quantity.sum()

In [366]:
from multiprocessing import Manager, Pool

manager = Manager()
shared_list = manager.list()

def do_stuff(x):
    global shared_list
    sets_num = [set_num for set_num in x]
    df_set = combine_sets(df_catalog_BOM,sets_num)
    score = compute_combination_fullfillment_score(df_6075_2_BOM,df_set)
    shared_list.append((sets_num,score))
    if len(shared_list)%1000==0:
        print len(shared_list)
    
pool = Pool(processes=10)
pool.map(do_stuff, COMBI_SETS)
pool.close()
        

3000
4000
4000


In [371]:
df_scores = pd.DataFrame([x for x in shared_list],columns=['sets','fullfilled_parts'])
df_sets = df_scores['sets'].apply(pd.Series)
df_sets.columns = ['set1', 'set2']
df_scores = pd.concat([df_scores,df_sets],axis=1)
#df_scores.drop(['sets'],axis=1,inplace=True)

In [374]:
df_scores.sort_values(by='fullfilled_parts',ascending=False)

,sets,fullfilled_parts,set1,set2
594,"[10654-1, 10255-1]",607.0,10654-1,10255-1
1135,"[10654-1, 75159-1]",607.0,10654-1,75159-1
2636,"[10654-1, 79014-1]",606.0,10654-1,79014-1
3162,"[10654-1, 10214-1]",604.0,10654-1,10214-1
277,"[10654-1, 10237-1]",603.0,10654-1,10237-1
2555,"[10654-1, 10223-1]",599.0,10654-1,10223-1
2757,"[10654-1, 75827-1]",598.0,10654-1,75827-1
2288,"[10654-1, 9474-1]",596.0,10654-1,9474-1
812,"[10654-1, 70810-1]",596.0,10654-1,70810-1
3678,"[10654-1, 10217-1]",595.0,10654-1,10217-1


In [414]:
df_sets = combine_sets(df_SETS_BOM,['10654-1','31044-1'])
df_matching = pd.merge(df_6075_2_BOM,df_sets,on='part_num',how='left',suffixes=['_6075_2','_10654_1'])
df_matching = pd.merge(df_matching,df_PARTS,on='part_num')
df_matching.quantity_10654_1.fillna(0,inplace=True)
df_matching['Missing_parts']=np.where((df_matching.quantity_6075_2-df_matching.quantity_10654_1)>0,df_matching.quantity_6075_2-df_matching.quantity_10654_1,0)
df_matching.sort_values(by='Missing_parts')

,part_num,quantity_6075_2,quantity_10654_1,name_part,part_cat_id,id,name_cat,Missing_parts
10,3023,13,29.0,Plate 1 x 2,14,14,Plates,0.0
15,3039,12,44.0,Slope 45° 2 x 2,3,3,Bricks Sloped,0.0
37,3710,4,10.0,Plate 1 x 4,14,14,Plates,0.0
13,3034,2,10.0,Plate 2 x 8,14,14,Plates,0.0
12,3032,8,8.0,Plate 4 x 6,14,14,Plates,0.0
11,3024,8,17.0,Plate 1 x 1,14,14,Plates,0.0
27,3623,8,9.0,Plate 1 x 3,14,14,Plates,0.0
9,3022,18,28.0,Plate 2 x 2,14,14,Plates,0.0
8,3021,6,9.0,Plate 2 x 3,14,14,Plates,0.0
38,3795,8,10.0,Plate 2 x 6,14,14,Plates,0.0
